# Inventory Optimization for Retail - LSTM Model Development

This notebook focuses on building and training an LSTM model for time series forecasting of retail inventory demand.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# For data preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# For LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set plotting style
plt.style.use('seaborn-whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Load Preprocessed Data

First, let's load the preprocessed data from the previous notebook.

In [ ]:
# Load the training and testing data
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/test_data.csv')

# Convert date columns to datetime
train_data['Week_Start'] = pd.to_datetime(train_data['Week_Start'])
test_data['Week_Start'] = pd.to_datetime(test_data['Week_Start'])

# Display the first few rows of the training data
train_data.head()

## 2. Prepare Data for LSTM

LSTM models require data to be in a specific format. We need to:
1. Scale the data
2. Create sequences of past observations
3. Reshape the data for LSTM input (samples, time steps, features)

In [ ]:
# Function to prepare data for a specific product-store combination
def prepare_lstm_data(product_id, store_id, sequence_length=4):
    # Filter data for the specific product and store
    train_product_store = train_data[(train_data['Product_ID'] == product_id) & 
                                    (train_data['Store_ID'] == store_id)].sort_values('Week_Start')
    test_product_store = test_data[(test_data['Product_ID'] == product_id) & 
                                   (test_data['Store_ID'] == store_id)].sort_values('Week_Start')
    
    # Select features
    features = ['Sales_Quantity', 'Inventory_Level', 'Sales_Lag_1', 'Sales_Lag_2', 'Sales_Lag_3', 'Sales_Lag_4',
                'Sales_Rolling_2', 'Sales_Rolling_4', 'Sales_Rolling_8']
    
    # Scale the features
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train_product_store[features])
    test_scaled = scaler.transform(test_product_store[features])
    
    # Create sequences
    X_train, y_train = create_sequences(train_scaled, sequence_length)
    X_test, y_test = create_sequences(test_scaled, sequence_length)
    
    return X_train, y_train, X_test, y_test, scaler, train_product_store, test_product_store

# Function to create sequences
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length, 0])  # Target is Sales_Quantity
    return np.array(X), np.array(y)

## 3. Build LSTM Model

Now, let's build an LSTM model for time series forecasting.

In [ ]:
# Function to build LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    
    # First LSTM layer with return sequences
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    
    # Second LSTM layer
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    
    # Dense output layer
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

## 4. Train and Evaluate Model

Let's train the model for a specific product-store combination and evaluate its performance.

In [ ]:
# Select a product-store combination for demonstration
product_id = train_data['Product_ID'].unique()[0]
store_id = train_data['Store_ID'].unique()[0]

# Prepare data
sequence_length = 4
X_train, y_train, X_test, y_test, scaler, train_product_store, test_product_store = prepare_lstm_data(
    product_id, store_id, sequence_length
)

# Build model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)

# Display model summary
model.summary()

In [ ]:
# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('../models/lstm_model.h5', save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

In [ ]:
# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('../images/model_loss.png')
plt.show()

## 5. Make Predictions

Now, let's use the trained model to make predictions on the test data.

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values
# Create dummy arrays with the same shape as the original scaled data
y_pred_dummy = np.zeros((len(y_pred), scaler.n_features_in_))
y_pred_dummy[:, 0] = y_pred.flatten()  # Set the first column (Sales_Quantity) to the predicted values

y_test_dummy = np.zeros((len(y_test), scaler.n_features_in_))
y_test_dummy[:, 0] = y_test  # Set the first column (Sales_Quantity) to the actual values

# Inverse transform
y_pred_inv = scaler.inverse_transform(y_pred_dummy)[:, 0]
y_test_inv = scaler.inverse_transform(y_test_dummy)[:, 0]

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label='Actual Sales')
plt.plot(y_pred_inv, label='Predicted Sales')
plt.title(f'Actual vs Predicted Sales for {product_id} at {store_id}')
plt.xlabel('Week')
plt.ylabel('Sales Quantity')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('../images/actual_vs_predicted.png')
plt.show()

## 6. Forecast Future Demand

Let's forecast the demand for the next 4 weeks.

In [ ]:
# Function to forecast future demand
def forecast_future_demand(model, last_sequence, scaler, n_steps=4):
    # Make a copy of the last sequence
    curr_sequence = last_sequence.copy()
    future_predictions = []
    
    for _ in range(n_steps):
        # Predict the next value
        pred = model.predict(curr_sequence.reshape(1, curr_sequence.shape[0], curr_sequence.shape[1]))
        
        # Create a dummy row with the prediction
        dummy_row = np.zeros((1, scaler.n_features_in_))
        dummy_row[0, 0] = pred[0, 0]  # Set Sales_Quantity
        
        # Inverse transform to get the actual value
        pred_inv = scaler.inverse_transform(dummy_row)[0, 0]
        future_predictions.append(pred_inv)
        
        # Update the sequence for the next prediction
        # Shift the sequence and add the new prediction
        new_row = curr_sequence[-1].copy()
        new_row[0] = pred[0, 0]  # Update Sales_Quantity
        # Update lag values (simplified approach)
        new_row[2] = curr_sequence[-1, 0]  # Sales_Lag_1 = previous Sales_Quantity
        new_row[3] = curr_sequence[-1, 2]  # Sales_Lag_2 = previous Sales_Lag_1
        new_row[4] = curr_sequence[-1, 3]  # Sales_Lag_3 = previous Sales_Lag_2
        new_row[5] = curr_sequence[-1, 4]  # Sales_Lag_4 = previous Sales_Lag_3
        # Update rolling averages (simplified approach)
        new_row[6] = (curr_sequence[-1, 0] + pred[0, 0]) / 2  # Sales_Rolling_2
        new_row[7] = (curr_sequence[-1, 0] + curr_sequence[-2, 0] + curr_sequence[-3, 0] + pred[0, 0]) / 4  # Sales_Rolling_4
        new_row[8] = curr_sequence[-1, 8]  # Keep Sales_Rolling_8 the same (simplified)
        
        curr_sequence = np.vstack([curr_sequence[1:], new_row])
    
    return future_predictions

# Get the last sequence from the test data
last_sequence = X_test[-1]

# Forecast future demand
future_predictions = forecast_future_demand(model, last_sequence, scaler, n_steps=4)

# Create dates for the future predictions
last_date = test_product_store['Week_Start'].iloc[-1]
future_dates = [last_date + pd.Timedelta(weeks=i+1) for i in range(len(future_predictions))]

# Display the forecasted demand
forecast_df = pd.DataFrame({
    'Week_Start': future_dates,
    'Forecasted_Sales': future_predictions
})
print("Forecasted Sales for the Next 4 Weeks:")
print(forecast_df)

In [ ]:
# Plot historical and forecasted sales
plt.figure(figsize=(15, 6))

# Historical data
historical_dates = list(train_product_store['Week_Start']) + list(test_product_store['Week_Start'])
historical_sales = list(train_product_store['Sales_Quantity']) + list(test_product_store['Sales_Quantity'])
plt.plot(historical_dates, historical_sales, label='Historical Sales', color='blue')

# Forecasted data
plt.plot(future_dates, future_predictions, label='Forecasted Sales', color='red', linestyle='--', marker='o')

# Add vertical line to separate historical and forecasted data
plt.axvline(x=last_date, color='gray', linestyle='--')
plt.text(last_date, max(historical_sales), 'Forecast Start', ha='right', va='top')

plt.title(f'Historical and Forecasted Sales for {product_id} at {store_id}')
plt.xlabel('Date')
plt.ylabel('Sales Quantity')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../images/historical_and_forecasted_sales.png')
plt.show()

## 7. Calculate Optimal Inventory Levels

Now, let's calculate the optimal inventory levels based on the forecasted demand.

In [ ]:
# Function to calculate optimal inventory levels
def calculate_optimal_inventory(forecasted_demand, safety_stock_factor=1.5, lead_time_days=3):
    # Convert lead time from days to weeks (assuming 7 days per week)
    lead_time_weeks = lead_time_days / 7
    
    # Calculate optimal inventory levels
    optimal_inventory = []
    for demand in forecasted_demand:
        # Base inventory = forecasted demand + safety stock
        base_inventory = demand * (1 + safety_stock_factor * lead_time_weeks)
        optimal_inventory.append(round(base_inventory))
    
    return optimal_inventory

# Calculate optimal inventory levels
optimal_inventory = calculate_optimal_inventory(future_predictions)

# Add to the forecast dataframe
forecast_df['Optimal_Inventory'] = optimal_inventory
print("Forecasted Sales and Optimal Inventory Levels:")
print(forecast_df)

In [ ]:
# Plot forecasted sales and optimal inventory levels
plt.figure(figsize=(12, 6))

# Bar chart for forecasted sales
plt.bar(forecast_df['Week_Start'], forecast_df['Forecasted_Sales'], color='skyblue', label='Forecasted Sales')

# Line chart for optimal inventory
plt.plot(forecast_df['Week_Start'], forecast_df['Optimal_Inventory'], color='red', marker='o', label='Optimal Inventory')

plt.title(f'Forecasted Sales and Optimal Inventory for {product_id} at {store_id}')
plt.xlabel('Week')
plt.ylabel('Quantity')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../images/forecasted_sales_and_optimal_inventory.png')
plt.show()

## 8. Save Results

Let's save our forecasting results and model for future use.

In [ ]:
# Create a directory for models if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the model
model.save('../models/lstm_model.h5')
print("Model saved to '../models/lstm_model.h5'")

# Save the forecast results
forecast_df.to_csv('../data/forecast_results.csv', index=False)
print("Forecast results saved to '../data/forecast_results.csv'")

## 9. Summary

In this notebook, we have:
1. Loaded and prepared the preprocessed data for LSTM modeling
2. Built and trained an LSTM model for time series forecasting
3. Evaluated the model's performance on test data
4. Forecasted future demand for the next 4 weeks
5. Calculated optimal inventory levels based on the forecasted demand
6. Visualized the results and saved them for future use

The model can now be used to forecast demand and optimize inventory levels for any product-store combination.